In [1]:
import pandas as pd
import numpy as np
import pickle, time, math
from datetime import datetime
import os 
import csv

In [2]:
#data=pd.read_csv('train_data.csv',index_col='index')
data=pd.read_csv('train_data.csv')
if 'totals_transactionRevenue' in data.columns:
    y=data.pop('totals_transactionRevenue').to_frame()
x=data.copy()

C:\Users\h_agu\Desktop\machine_learning\Miniconda3\envs\tensorflow-gpu3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
x=x.drop(['visitStartTime','visitNumber','visitId','totals_hits','totals_pageviews','sessionId','fullVisitorId','date'],axis=1)

In [4]:
for col in x.columns:
    if 'visitStartTime_' in col:
        x[col]=x[col].astype(np.object)
    if 'date_' in col:
        x[col]=x[col].astype(np.object)

In [5]:
features=x.columns.tolist()
category_features=[]
category_features_idx=[]
for col in features:
    if x[col].dtypes==np.object:
        #print(col,np.object)
        category_features.append(col)
        category_features_idx.append(features.index(col))
    if x[col].dtypes==np.bool:
        #print(col,np.bool)
        category_features.append(col)
        category_features_idx.append(features.index(col))

In [6]:
from catboost import CatBoostClassifier,Pool
from sklearn.model_selection import StratifiedKFold, train_test_split

In [7]:
#xtrain,xtest,ytrain,ytest=train_test_split(x, y, test_size=0.1, random_state=42, shuffle=True, stratify=y)
_y=y.copy()

In [8]:
_y.loc[(_y['totals_transactionRevenue']>0),'totals_transactionRevenue']=1
_y.loc[(_y['totals_transactionRevenue']==0),'totals_transactionRevenue']=0

In [10]:
features=x.columns.tolist()
category_features=[]
category_features_idx=[]
for col in features:
    if x[col].dtype == np.object or x[col].dtype == np.bool:
        category_features.append(col)
        category_features_idx.append(features.index(col))

In [11]:
from hyperopt import fmin, tpe, Trials, STATUS_OK, STATUS_FAIL, hp, pyll

## binary classification

In [9]:
xtrain,xtest,ytrain,ytest=train_test_split(x, _y, test_size=0.4, random_state=42, shuffle=True, stratify=_y)

In [11]:
sF=StratifiedKFold(n_splits=5,shuffle=True)
cv_split=[]
for train_idx,test_idx in sF.split(xtrain,ytrain):
    pool_train=Pool(xtrain.iloc[train_idx],ytrain.iloc[train_idx],cat_features=category_features_idx)
    pool_test=Pool(xtrain.iloc[test_idx],ytrain.iloc[test_idx],cat_features=category_features_idx)
    cv_split.append((pool_train,pool_test))

In [13]:
param_space = {
            'depth': hp.choice('depth', [4,6]),
            'border_count': hp.choice('border_count', [64,128]),
            'learning_rate': hp.loguniform('learning_rate', -5, 0),
            'random_strength': hp.choice('random_strength', [1,  20]),
            'one_hot_max_size': hp.choice('one_hot_max_size', [5, 25, 225]),
            'l2_leaf_reg': hp.loguniform('l2_leaf_reg', 0, np.log(10)),
            'bagging_temperature': hp.uniform('bagging_temperature', 0, 1),
            'leaf_estimation_iterations':hp.choice('leaf_estimation_iterations',[1,3,5,7,10]),
            'max_ctr_complexity':hp.quniform('max_ctr_complexity',1,5,1),
            'leaf_estimation_method':hp.choice('leaf_estimation_method',['Newton','Gradient']),
            #'class_weights': (hp.choice('non_class_weights_ratio',[1]), hp.uniform('class_weights_ratio',1,20))
        }

In [14]:
def fit(params=None, dtrain=None, dtest=None, n_estimators=None, seed=0, run_time=None, run_cv_id=0, eval_no=0, verbose=False):
    global metric,column_names
    #print(run_cv_id, eval_no)
    path="./cv_run/"+str(run_time)
    if not os.path.isdir(path):
        os.mkdir(path)
    fpath=path+"/"+str(eval_no)+"."+str(run_cv_id)
    if not os.path.isdir(fpath):
        os.mkdir(fpath)    
    params.update({"iterations": n_estimators})
    params.update({"eval_metric": metric})
    params.update({"logging_level": 'Verbose'})
    params.update({"metric_period": 100})
    params.update({"random_seed": seed})
    #params.update({"leaf_estimation_method": "Newton"})
    #params.update({"leaf_estimation_iterations" : 10})
    params.update({"rsm" : 1})
    params.update({"thread_count" : 8})
    params.update({"fold_len_multiplier": 2})
    #params.update({"max_ctr_complexity":5})
    params.update({"train_dir": fpath})
    params.update({"calc_feature_importance" : True})
    params.update({'od_type':'Iter'})
    params.update({'od_wait':30})
    
    bst = CatBoostClassifier(**params)
    bst.fit(dtrain, eval_set=dtest, use_best_model=True)
    with open(fpath + "/test_error.tsv", "r") as f:
        reader=np.array(list(csv.reader(f,delimiter='\t'))).squeeze()
    header=reader[0]
    feature=dict()
    for col, val in zip(features,bst.__dict__['_feature_importance']):
        feature.update({col:val})
    #pd.to_pickle(bst.__dict__['_feature_importance'],path+"/feature_importance."+str(eval_no)+"."+str(run_cv_id))
    pd.to_pickle(feature,path+"/feature_importance."+str(eval_no)+"."+str(run_cv_id))
    idx=(header==metric).argmax()
    #print('idx',idx, metric)
    results=(reader[1:reader.shape[0],idx]).astype(np.float)
    
    if metric=='AUC' or metric=='Accuracy':
        #print("metric",metric)
        results=1-results
    #print('results',results)
    return bst, results

In [45]:
def mean_each_iter(results):
    #global metric
    lengs=[]
    _=[lengs.append(len(res)) for res in results]
    #if metric in ['AUC','Accuracy']:
    mlen=np.max(lengs)
    #else:
    #    mlen=np.min(lengs)
    #print(lengs,mlen)
    a=[]
    for run in results:
        a.append((np.pad(run,(0,mlen-len(run)),'constant')).tolist())
    x=np.array(a)
    #print(x.shape)
    means=[]
    for i in range(x.shape[1]):
        mean=0
        count=0
        for j in range(x.shape[0]):
            if x[j,i] > 0:
                count=count+1
                mean=mean+x[j,i]
        means.append(mean/count)
    return means

In [46]:
def run_cv(cv_pairs, params=None, n_est=None, verbose=False, run_time=None):
    global default_params,n_estimators,best_loss,hyperopt_eval_num,metric,hyperopt_evals,metric
    params = params or default_params
    n_estimators = n_est or n_estimators
    #print('run_cv',hyperopt_eval_num)
    evals_results, start_time = [], time.time()
    _loss=[]
    i=0
    for dtrain, dtest in cv_pairs:
        _, evals_result = fit(params, dtrain, dtest, n_estimators, run_time=run_time, run_cv_id=i, eval_no=hyperopt_eval_num+1)
        #evals_results.append(np.mean(evals_result,axis=0))
        evals_results.append(evals_result)
        _loss.append(np.min(evals_result))
        i=i+1
    
    mean_evals_results = mean_each_iter(evals_results)
    best_n_estimators = np.argmin(mean_evals_results) + 1
    eval_time = time.time() - start_time
    
    
    cv_result = {'loss': mean_evals_results[best_n_estimators - 1] ,
                 'best_n_estimators': best_n_estimators, 
                 'eval_time': eval_time,
                 'status': STATUS_FAIL if np.isnan(mean_evals_results[best_n_estimators - 1]) else STATUS_OK,
                 'params': params.copy(),
                 'losses': _loss
                }
    best_loss = min(best_loss, cv_result['loss'])
    hyperopt_eval_num += 1
    cv_result.update({'hyperopt_eval_num': hyperopt_eval_num, 'best_loss': best_loss})
        
    if verbose:
        print ('[{0}/{1}]\teval_time={2:.2f} sec\tcurrent_{3}={4:.6f}\tmin_{3}={5:.6f}'.format(
                    hyperopt_eval_num, hyperopt_evals, eval_time,
                    metric, cv_result['loss'], best_loss))
    return cv_result

In [19]:
n_estimators=1000
max_evals = 20
hyperopt_evals=max_evals
metric="Accuracy"

this_trials = Trials()
run_time=(datetime.now()).strftime('%Y%m%d%H%M')
hyperopt_eval_num, best_loss, split_pair_data = 0, np.inf, None
args=param_space
_ = fmin(fn=lambda args: run_cv(cv_split, params=args, n_est=n_estimators, verbose=True,run_time=run_time), 
         space=args, algo=tpe.suggest, max_evals=max_evals, trials=this_trials)

with open('./cv_run/'+run_time+'/trails.pickle','wb') as f:
    pickle.dump(this_trials,f)
    

metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
[1/20]	eval_time=533.12 sec	current_Accuracy=0.011924	min_Accuracy=0.011924
metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
[2/20]	eval_time=116.37 sec	current_Accuracy=0.012483	min_Accuracy=0.011924
metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
[3/20]	eval_time=331.41 sec	current_Accuracy=0.012248	min_Accuracy=0.011924
metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
[4/20]	eval_time=143.33 sec	current_Accuracy=0.012106	min_Accuracy=0.011924
metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
[5/20]	eval_time=355.88 sec	current_Accuracy=0.012194	min_Accuracy=0.011924
metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
[6/20]	eval_time=50.67 sec	current_Accuracy=0.012743	min_Accuracy=0.011924
metric Accuracy
metric Accuracy
metric Accuracy
metric Accuracy
m

In [17]:
with open('cv_run/201810041600/trails.pickle', 'rb') as f:
    trials=pickle.load(f)

In [39]:
metric='Accuracy'
arg=trials.trials[3]['result']['params']
run_time=(datetime.now()).strftime('%Y%m%d%H%M')
trainP=Pool(xtrain,np.log1p(ytrain),cat_features=category_features_idx)
testP=Pool(xtest,np.log1p(ytest),cat_features=category_features_idx)
bst,res=fit(params=arg,dtrain=trainP,dtest=testP,n_estimators=2000,seed=13,run_time=run_time)

KeyboardInterrupt: 

In [57]:
#with open('./model/model.json','wb') as f:
bst.save_model('./model/model_class.cbm')

In [58]:
loadbst=bst.load_model('./model/model_class.cbm')

## Regression

In [48]:
from catboost import CatBoostRegressor

In [12]:
xtrain,xtest,ytrain,ytest=train_test_split(x, y, test_size=0.4, random_state=42, shuffle=True, stratify=_y)

In [41]:
y_t=_y.totals_transactionRevenue.loc[ytrain.index.to_series()]
sF=StratifiedKFold(n_splits=5,shuffle=True)
cv_split=[]
for train_idx,test_idx in sF.split(xtrain,y_t):
    pool_train_log=Pool(xtrain.iloc[train_idx],np.log1p(ytrain.iloc[train_idx]),cat_features=category_features_idx)
    pool_test_log=Pool(xtrain.iloc[test_idx],np.log1p(ytrain.iloc[test_idx]),cat_features=category_features_idx)
    cv_split.append((pool_train_log,pool_test_log))

In [42]:
param=dict(
{'bagging_temperature': 0.334998678341902,
 'border_count': 128,
 'depth': 4,
 'l2_leaf_reg': 7.201514933380306,
 'leaf_estimation_iterations': 5,
 'leaf_estimation_method': 'Gradient',
 'learning_rate': 0.8915171912524651,
 'max_ctr_complexity': 4.0,
 'one_hot_max_size': 225,
 'random_strength': 1,
 'iterations': 2001,
 "logging_level": 'Verbose',
 "metric_period": 100,
 #'eval_metric': 'Accuracy',
 'logging_level': 'Silent',
 'random_seed': 0,
 'rsm': 1,
 'thread_count': 8,
 'fold_len_multiplier': 2,
 'calc_feature_importance': True,
 'od_type': 'Iter',
 'od_wait': 30})

In [63]:
param_space = {
            'depth': hp.choice('depth', [4,6,8]),
            'border_count': hp.choice('border_count', [32,64,128]),
            'learning_rate': hp.loguniform('learning_rate', -5, 0),
            'random_strength': hp.choice('random_strength', [1, 5, 10, 20]),
            'one_hot_max_size': hp.choice('one_hot_max_size', [5, 25, 225]),
            'l2_leaf_reg': hp.loguniform('l2_leaf_reg', 0, np.log(10)),
            'bagging_temperature': hp.uniform('bagging_temperature', 0, 1),
            'leaf_estimation_iterations':hp.choice('leaf_estimation_iterations',[1,3,5,7,10]),
            'max_ctr_complexity':hp.quniform('max_ctr_complexity',1,5,1),
            'leaf_estimation_method':hp.choice('leaf_estimation_method',['Newton','Gradient']),
            'rsm':hp.uniform('rsm',0,1),
            'fold_len_multiplier':hp.choice('fold_len_multiplier',[1,2,3,4])
            #'class_weights': (hp.choice('non_class_weights_ratio',[1]), hp.uniform('class_weights_ratio',1,20))
        }

In [66]:
def fit(params=None, dtrain=None, dtest=None, n_estimators=None, seed=0, run_time=None, run_cv_id=0, eval_no=0, verbose=False):
    global metric,column_names
    #print(run_cv_id, eval_no)
    path="./cv_run/"+str(run_time)
    if not os.path.isdir(path):
        os.mkdir(path)
    fpath=path+"/"+str(eval_no)+"."+str(run_cv_id)
    if not os.path.isdir(fpath):
        os.mkdir(fpath)    
    params.update({"iterations": n_estimators})
    params.update({"eval_metric": metric})
    params.update({"logging_level": 'Silent'})
    params.update({"metric_period": 100})
    params.update({"random_seed": seed})
    #params.update({"rsm" : 1})
    params.update({"thread_count" : 8})
    #params.update({"fold_len_multiplier": 2})
    params.update({"train_dir": fpath})
    params.update({"calc_feature_importance" : True})
    params.update({'od_type':'Iter'})
    params.update({'od_wait':30})
    bst = CatBoostRegressor(**params)
    bst.fit(dtrain, eval_set=dtest, use_best_model=True)
    with open(fpath + "/test_error.tsv", "r") as f:
        reader=np.array(list(csv.reader(f,delimiter='\t'))).squeeze()
    header=reader[0]
    feature=dict()
    for col, val in zip(features,bst.__dict__['_feature_importance']):
        feature.update({col:val})
    #pd.to_pickle(bst.__dict__['_feature_importance'],path+"/feature_importance."+str(eval_no)+"."+str(run_cv_id))
    pd.to_pickle(feature,path+"/feature_importance."+str(eval_no)+"."+str(run_cv_id))
    idx=(header==metric).argmax()
    #print('idx',idx, metric)
    results=(reader[1:reader.shape[0],idx]).astype(np.float)
    
    if metric=='AUC' or metric=='Accuracy':
        #print("metric",metric)
        results=1-results
    #print('results',results)
    return bst, results

In [67]:
n_estimators=10
max_evals = 2
hyperopt_evals=max_evals
metric="RMSE"

this_trials = Trials()
run_time=(datetime.now()).strftime('%Y%m%d%H%M')
hyperopt_eval_num, best_loss, split_pair_data = 0, np.inf, None
args=param_space
_ = fmin(fn=lambda args: run_cv(cv_split, params=args, n_est=n_estimators, verbose=True,run_time=run_time), 
         space=args, algo=tpe.suggest, max_evals=max_evals, trials=this_trials)

with open('./cv_run/'+run_time+'/trails.pickle','wb') as f:
    pickle.dump(this_trials,f)

[1/2]	eval_time=18.84 sec	current_RMSE=1.812583	min_RMSE=1.812583
[2/2]	eval_time=27.98 sec	current_RMSE=1.726956	min_RMSE=1.726956


In [12]:
trainP=Pool(xtrain,np.log1p(ytrain),cat_features=category_features_idx)
testP=Pool(xtest,np.log1p(ytest),cat_features=category_features_idx)